In [1]:
import numpy as np
import pandas as pd
import time
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import joblib
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import matplotlib
import os

%load_ext autoreload
%autoreload 2
# %matplotlib notebook
%matplotlib inline

# Data Preprocessing

In [2]:
# body_part = "head"
# file_names = ['./training_data/head_train_1_to_6.log']

# body_part = "shoulder_right"
# file_names = [
#              './training_data/shoulder_right_train_10_17.log',
#              './training_data_old/shoulder_right_data_6.log',
#              './training_data_old/shoulder_right_data_7.log',
#              './training_data_old/shoulder_right_data_8.log',
#              './training_data_old/shoulder_right_data_9.log',
#             ]

body_part = "shoulder_left"
# file_names = ['./training_data/shoulder_left_train_2.log',
#               './training_data/shoulder_left_train_5_6_7_9_10.log'
#              ]
file_names = [
    './training_data/train_left_new_tp.bag',
    './training_data/train_left_new.bag',
    './training_data/train_left_new_2.bag',
    './training_data/val_fb_fast_gt.csv'
]

val_file_names = [
#     './training_data/val_fb_fast_gt.csv'
    './training_data/val_fb_sfast_dvbf_medium.bag'
]

In [3]:
# orig_dataset = pd.concat([pd.read_csv(f, header=0) for f in file_names])
# orig_dataset = orig_dataset.values[1:len(orig_dataset)-2,1:]
# dataset = [data[(np.abs(stats.zscore(data[["roll", "pitch", "yaw"]])) < 2.75).all(axis=1)] for data in dataset]

abnormal_threshold = 0.5
dataset = []
orig_dataset = []

euler_idxs = ["position_0", "position_1", "position_2"]
action_idxs = ["target_position_0", "target_position_1", "target_position_2"]
magnetic_idxs = [f'{coord}_{id}' for id in range(4) for coord in ['x','y','z']]

for name in file_names:
    df = pd.read_csv(name, header=0)
    
    df['date'] = pd.to_datetime(df.Time, unit='s')
    df.groupby(pd.Grouper(key='date', freq='20L')).mean().interpolate()
    
    orig_dataset.append(df.copy())
    
    # interpolate nan outputs (loss tracking)
    for c in euler_idxs:
        df[c] = df[c].interpolate()
    
    # interpolate abnormal inputs
    for c in magnetic_idxs:
        bad_idx = df.index[df[c].pct_change().abs().ge(abnormal_threshold)]
        df.loc[bad_idx, c] = np.nan
        df[c] = df[c].interpolate()
    
    dataset.append(df)

dataset_len = [len(data) for data in dataset]
dataset = pd.concat(dataset) 
orig_dataset = pd.concat(orig_dataset) 

print(f'{np.sum(dataset_len)} values')

dataset = dataset[["Time"] + magnetic_idxs + euler_idxs + action_idxs].values[:,1:]
orig_dataset = orig_dataset[["Time"] + magnetic_idxs + euler_idxs + action_idxs].values[:,1:]

euler_set = dataset[:, 12:15]
action_set = dataset[:, 15:]
sensors_set = dataset[:, :12]
orig_sensors_set = orig_dataset[:, :12]
print(f'max euler {str(np.amax(euler_set))}')
print(f'min euler {str(np.amin(euler_set))}')

238476 values
max euler 3.135378403228545
min euler -3.1407292575036365


In [4]:
val_dataset = []
val_orig_dataset = []

for name in val_file_names:
    df = pd.read_csv(name, header=0)
    
    val_orig_dataset.append(df.copy())
    
    # interpolate nan outputs (loss tracking)
    for c in euler_idxs:
        df[c] = df[c].interpolate()
        
    val_dataset.append(df)
    
val_dataset_len = [len(data) for data in val_dataset]
val_dataset = pd.concat(val_dataset) 
val_orig_dataset = pd.concat(val_orig_dataset)

print(f'{np.sum(val_dataset_len)} values')

val_dataset = val_dataset[["Time"] + magnetic_idxs + euler_idxs + action_idxs].values[:,1:]
val_orig_dataset = val_orig_dataset[["Time"] + magnetic_idxs + euler_idxs + action_idxs].values[:,1:]

val_euler_set = val_dataset[:, 12:15]
val_action_set = val_dataset[:, 15:]
val_sensors_set = val_dataset[:, :12]
val_orig_sensors_set = val_orig_dataset[:, :12]
print(f'max euler {str(np.amax(euler_set))}')
print(f'min euler {str(np.amin(euler_set))}')

15047 values
max euler 3.135378403228545
min euler -3.1407292575036365


## Orientation preprocessing 

In [5]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from libs.orientation_utils import compute_rotation_matrix_from_euler, compute_ortho6d_from_rotation_matrix, compute_rotation_matrix_from_ortho6d, compute_euler_angles_from_rotation_matrices

In [6]:
euler_set_in = np.zeros_like(euler_set)
euler_set_in[:, 0] = euler_set[:, 2] 
euler_set_in[:, 1] = euler_set[:, 1] 
euler_set_in[:, 2] = euler_set[:, 0] 

euler_set = torch.Tensor(euler_set_in).cuda()
rot_set = compute_rotation_matrix_from_euler(euler_set)
rot_set = compute_ortho6d_from_rotation_matrix(rot_set).cpu().detach().numpy()

In [7]:
val_euler_set_in = np.zeros_like(val_euler_set)
val_euler_set_in[:, 0] = val_euler_set[:, 2] 
val_euler_set_in[:, 1] = val_euler_set[:, 1] 
val_euler_set_in[:, 2] = val_euler_set[:, 0] 

val_euler_set = torch.Tensor(val_euler_set_in).cuda()
val_rot_set = compute_rotation_matrix_from_euler(val_euler_set)
val_rot_set = compute_ortho6d_from_rotation_matrix(val_rot_set).cpu().detach().numpy()

## Normalization

In [8]:
sensors_scaler = MinMaxScaler(feature_range=(-1., 1.))
action_scaler = MinMaxScaler(feature_range=(-1., 1.))

# Split magnetic sensors into 4 independent distributions again
orig_sensors_set = sensors_scaler.fit_transform(orig_sensors_set).reshape(-1, 4, 3)
sensors_set = sensors_scaler.transform(sensors_set).reshape(-1, 4, 3)
action_set = action_scaler.fit_transform(action_set)

In [9]:
val_orig_sensors_set = sensors_scaler.transform(val_orig_sensors_set).reshape(-1, 4, 3)
val_sensors_set = sensors_scaler.transform(val_sensors_set).reshape(-1, 4, 3)
val_action_set = action_scaler.transform(val_action_set)

## Torch dataloader

In [10]:
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers.csv_logs import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from libs.data_modules import SensorsDataModule
from libs.nn_models import mse_seq
from libs.dvbf_models import DVBF

In [11]:
look_back = 50

data_in = []
data_u = []
data_hat_in = []
data_out = []

start_idx = 0
for l in dataset_len:
    # Ignore the last batch
    for i in range(start_idx, start_idx+l-look_back*2, 10): # step with look_back
        data_in.append(orig_sensors_set[i:i+look_back])
        data_hat_in.append(sensors_set[i:i+look_back])
        data_u.append(action_set[i:i+look_back])
        data_out.append(rot_set[i+1:i+look_back+1])
    print(len(data_in))
    start_idx += l

784
10732
21418
23809


In [12]:
val_data_in = []
val_data_u = []
val_data_hat_in = []
val_data_out = []

start_idx = 0
for l in val_dataset_len:
    # Ignore the last batch
    for i in range(start_idx, start_idx+l-look_back*2, 10): # step with look_back
        val_data_in.append(val_orig_sensors_set[i:i+look_back])
        val_data_hat_in.append(val_sensors_set[i:i+look_back])
        val_data_u.append(val_action_set[i:i+look_back])
        val_data_out.append(val_rot_set[i+1:i+look_back+1])
    print(len(val_data_in))
    start_idx += l

1495


# Training

In [13]:
p = dict(
    seq_len = look_back,
    batch_size = 1000, 
    max_epochs = 10000,
    n_frames = 4,
    n_observations = 3,
    n_actions = 3,
    n_latents = 32, # 16
    n_outputs = 6,
    n_initial_obs = 5,
    hidden_size = 100,
    learning_rate = 5e-4,
    alpha = 30.0, # 10.0
    beta = 0.1,
    temperature = 1e5
)

In [14]:
# model_path = f"./outputs_idp/{body_part}_dvbf_long_trajs_ad_with_target_rot6D"
model_path = f"./outputs_idp_isrr/{body_part}_dvbf_long_trajs_no_ad_with_target_rot6D"

if not os.path.exists(model_path):
    os.makedirs(model_path)

joblib.dump((sensors_scaler, action_scaler), f'{model_path}/scaler.pkl')
with open(f'{model_path}/hyperparams.json', 'w') as fp:
    json.dump(p, fp)

In [15]:
seed_everything(1)

checkpoint_callback = ModelCheckpoint(
    monitor="val_mse",
    dirpath=model_path,
    filename=f"best-alpha={p['alpha']}-reg=0.01",
#     filename="best-{epoch:03d}-{val_mse:.5f}",
#     save_top_k=3,
    mode="min",
)

early_stop_callback = EarlyStopping(monitor="val_mse", min_delta=0.00, patience=100, verbose=False, mode="min")

trainer = Trainer(
    max_epochs=p['max_epochs'],
    callbacks=[checkpoint_callback, early_stop_callback],
    gpus=1,
    log_every_n_steps=10,
    progress_bar_refresh_rate=2,
#     resume_from_checkpoint=f"{model_path}/best-epoch=3592-val_loss=-210.53926.ckpt"
)

model = DVBF(
    n_frames = p['n_frames'],
    n_observations = p['n_observations'],
    n_actions = p['n_actions'],
    n_latents = p['n_latents'],
    n_outputs = p['n_outputs'],
    n_initial_obs = p['n_initial_obs'],
    hidden_size = p['hidden_size'],
    seq_len = p['seq_len'],
    batch_size = p['batch_size'],
    learning_rate = p['learning_rate'],
    alpha = p['alpha'],
    beta = p['beta'],
    temperature = p['temperature'],
)


dm = SensorsDataModule(
#     data_x = data_in,
    data_x = data_hat_in,
    data_u = data_u,
    data_y = data_out,
    data_x_hat = data_hat_in,
    data_x_val = val_data_in,
    data_u_val = val_data_u,
    data_y_val = val_data_out,
    seq_len = p['seq_len'],
    batch_size = p['batch_size']
)

trainer.fit(model, dm)
trainer.test(model, datamodule=dm)

Global seed set to 1
/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=2)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dr

Validation sanity check: 0it [00:00, ?it/s]

/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 1
/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/pyto

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_kl': 48.98530578613281,
 'test_loss': -443.1763916015625,
 'test_mse': 0.0059826369397342205,
 'test_nllx': -23.427471160888672,
 'test_nlly': -14.057823181152344}
--------------------------------------------------------------------------------


[{'test_loss': -443.1763916015625,
  'test_nllx': -23.427471160888672,
  'test_nlly': -14.057823181152344,
  'test_kl': 48.98530578613281,
  'test_mse': 0.0059826369397342205}]

In [16]:
trainer.test(model, datamodule=dm)

/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_kl': 48.997589111328125,
 'test_loss': -442.931396484375,
 'test_mse': 0.005861682817339897,
 'test_nllx': -23.400794982910156,
 'test_nlly': -14.050552368164062}
--------------------------------------------------------------------------------


/home/thobotics/anaconda3/envs/tum/lib/python3.6/site-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  f"DataModule.{name} has already been called, so it will not be called again. "


[{'test_loss': -442.931396484375,
  'test_nllx': -23.400794982910156,
  'test_nlly': -14.050552368164062,
  'test_kl': 48.997589111328125,
  'test_mse': 0.005861682817339897}]